<a href="https://colab.research.google.com/github/yuanfeiwo/test-firstrepository/blob/master/Module4_ML_in_Finance_Module_4_Case_study_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering

## Install required packages

In [1]:
install.packages('dplyr')
install.packages('cluster')
install.packages('Rtsne')
install.packages('ggplot2')
install.packages('RCurl')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘bitops’




## R Packages  imported to assist in the analysis

Below are the set of libraries imported into this python notebook that is useful in doing exploratory and predictive data analysis.

In [2]:
#  Module: Clustering Case Study 
#  Version 1.1
#  Topic : Clustering
#  Example source: https://www.kaggle.com/wendykan/lending-club-loan-data
#####################################################################
#  For support or questions, contact Sri Krishnamurthy at
#  sri@quantuniversity.com
#  Copyright 2016 QuantUniversity LLC.
#####################################################################

### R packages
## Data wrangling
# install.packages("dplry")
library(dplyr)

## Clustering
# install.packages("cluster")
library(cluster) # for gower similarity and pam

## Visualization 
# install.packages("Rtsne")
# install.packages("ggplot2")
library(Rtsne) # for t-SNE plot with dimension reduction 
library(ggplot2) # for visualization
# download data
library(RCurl)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## Step 1 Reading data

In [3]:
### Step 1 Reading data
url <- getURL("https://qusandbox-dataset.s3-us-west-2.amazonaws.com/LendingClubLoan.csv")
LendingClubLoan <- read.csv(textConnection(url))
LendingClubLoan_Clustering <- dplyr::select(LendingClubLoan, -loan_status_Binary)

# Randomly select 5000 rows
set.seed(33)
LendingClubLoan_Clustering <- LendingClubLoan_Clustering[sample(1:nrow(LendingClubLoan_Clustering),
                                                                5000, replace=FALSE), ]
str(LendingClubLoan_Clustering)

'data.frame':	5000 obs. of  16 variables:
 $ loan_amnt          : int  12000 22000 25000 5700 35000 16000 6500 5000 10000 2000 ...
 $ term               : chr  " 36 months" " 60 months" " 36 months" " 60 months" ...
 $ int_rate           : num  5.99 12.69 11.71 18.64 19.03 ...
 $ installment        : num  365 497 827 147 587 ...
 $ grade              : chr  "A" "B" "B" "E" ...
 $ sub_grade          : chr  "A2" "B5" "B3" "E1" ...
 $ emp_length         : chr  "3 years" "1 year" "10+ years" "10+ years" ...
 $ home_ownership     : chr  "MORTGAGE" "MORTGAGE" "RENT" "MORTGAGE" ...
 $ annual_inc         : num  36000 115000 70000 107004 99225 ...
 $ verification_status: chr  "Source Verified" "Verified" "Verified" "Source Verified" ...
 $ issue_d            : chr  "11-Aug" "11-Oct" "11-Dec" "11-Oct" ...
 $ purpose            : chr  "debt_consolidation" "debt_consolidation" "debt_consolidation" "other" ...
 $ addr_state         : chr  "MD" "TX" "NY" "NJ" ...
 $ dti                : num  12.73 0

## Step 2 Calculating distance
Use Gower's distance to measure the distance between two entity. The attribute of entity may contain combinations of logical, numerical, categorical or text data. The distance is calculated as a number between 0 (identical) and 1 (maximally dissimilar).

In [5]:
### Step 2 Calculating distance
gower_dist <- daisy(LendingClubLoan_Clustering, metric = "gower")

## Step 2-1 Checking attributes to ensure the correct methods are being used
# (I = interval, N = nominal)
summary(gower_dist)

ERROR: ignored

In [6]:
#which(gower_mat == min(gower_mat[gower_mat != min(gower_mat)]))

### Step 2-2 Outputting most similar pair

In [7]:
## Step 2-2 Outputting most similar pair
gower_mat <- as.matrix(gower_dist)

LendingClubLoan_Clustering[
  which(gower_mat == min(gower_mat[gower_mat != min(gower_mat)]),
        arr.ind = TRUE)[1, ], ]

ERROR: ignored

### Step 2-3 Outputting most dissimilar pair

In [ ]:
## Step 2-3 Outputting most dissimilar pair
LendingClubLoan_Clustering[
  which(gower_mat == max(gower_mat[gower_mat != max(gower_mat)]),
        arr.ind = TRUE)[1, ], ]

From the output, you can see that the most similar pair have similar loan information, such as loan amount, grade and interest rate, while the most dissimilar pair are very different from each other. By using Gower distance, you can find some relationships in data.

## Step 3 Selecting number of clusters 
One method to select the number of clusters is to compute the mean Silhouette Coefficient of all samples and select the number with best performance.

In [ ]:
### Step 3 Selecting number of clusters 
sil_width <- c(NA)

for(i in 2:10){
  pam_fit <- pam(gower_dist,
                 diss = TRUE,
                 k = i)
  
  sil_width[i] <- pam_fit$silinfo$avg.width
}

# Plot sihouette width (higher is better, based on the result, Cluster 3 yields the highest value)
plot(1:10, sil_width,
     xlab = "Number of clusters",
     ylab = "Silhouette Width")
lines(1:10, sil_width)

From the Silhouette Score plotting, the score drops dramatically from 3 to 4 clusters, where you can see the highest score. It seems like 3 is the optimal number of clusters.

## Step 4 Partitioning around medoids (PAM)
The PAM algorithm is very similar to K-means, mostly because both are partitional algorithms, in other words, they break the dataset into groups (= clustering), while also trying to minimize the error.

### Step 4-1 Summary of clustering results

In [ ]:
### Step 4 Partitioning around medoids (PAM) 
## Step 4-1 Summary of clustering results
pam_fit <- pam(gower_dist, diss = TRUE, k = 3)

pam_results <- LendingClubLoan_Clustering %>%
  mutate(cluster = pam_fit$clustering) %>%
  group_by(cluster) %>%
  do(the_summary = summary(.))

pam_results$the_summary

### Step 4-2 Center medoid of each cluster

In [ ]:
## Step 4-2 Exemplars of each cluster
LendingClubLoan_Clustering[pam_fit$medoids, ]

You can see that <b>the loan_amnt, int_rate, and grade</b> have significant difference among the clusters.

### Step 4-3 Visualization with reduced-dimension 

In [ ]:
## Step 4-3 Visualization with reduced-dimension 
tsne_obj <- Rtsne(gower_dist, is_distance = TRUE)

tsne_data <- tsne_obj$Y %>%
  data.frame() %>%
  setNames(c("X", "Y")) %>%
  mutate(cluster = factor(pam_fit$clustering))

ggplot(aes(x = X, y = Y), data = tsne_data) +
  geom_point(aes(color = cluster)) +
  ggtitle("Clusters on reduced-dimension")

### Step 4-4 Plotting clusters over two variables
Try to plot the clusters over several variables, and you will understand how the model separates the clusters.

For example, the variables you can use are "loan_anmt", "grade" and "int_rate". When the "loan_anmt" is picked for the x-axis, while "grade" and "int_rate" are picked for the y-axis, what can we tell from these graphs?

In [ ]:
## Step 4-4 Plotting clusters over two variables
Clustering <- LendingClubLoan_Clustering %>%
  mutate(Cluster = pam_fit$clustering)

ggplot(aes(x = loan_amnt, y = grade), data = Clustering) +
  geom_point(aes(color = factor(Cluster))) +
  ggtitle("Clusters on two variables")

ggplot(aes(x = loan_amnt, y = int_rate), data = Clustering) + 
  geom_point(aes(color = factor(Cluster))) +
  ggtitle("Clusters on two variables")

## Step 5 Agglomerative hierarchical clustering

### Step 5-1 Computes agglomerative hierarchical clustering
Compute the hierarchical clustering using "manhattan" metrics.

In [ ]:
### Step 5 Agglomerative hierarchical clustering
## Step 5-1 Computes agglomerative hierarchical clustering
agn1 <- agnes(gower_dist, metric = "manhattan", stand = FALSE)

In [ ]:
plot(agn1, main= "Hierarchical clustering")

From the dendrograms, find 6 clusters, where you see large subsets relatively.

### Step 5-2 Visualization on reduced-dimension 

In [ ]:
## Step 5-2 Visualization on reduced-dimension 
HierarchicalCluster <- LendingClubLoan_Clustering %>%
  mutate(Cluster=cutree(agn1,6))

tsne_obj <- Rtsne(gower_dist, is_distance = TRUE)


In [ ]:
tsne_data <- tsne_obj$Y %>%
  data.frame() %>%
  setNames(c("X", "Y")) %>%
  mutate(cluster = factor(cutree(agn1,6)))

ggplot(aes(x = X, y = Y), data = tsne_data) +
  geom_point(aes(color = cluster)) +
  ggtitle("Clusters on reduced-dimension Plot")

This model creates 6 clusters, 4 of which are large clusters in the plot. It seems like the other 2 clusters are just outliers compared with the large clusters. The data is clustered better than PAM clustering as clearer boundaries are shown between clusters when hierarchical clustering method is used.

### Step 5-3 Visualization on two variables

In [ ]:
## Step 5-3 Visualization on two variables
ggplot(aes(x = loan_amnt, y = grade), data = HierarchicalCluster) +
  geom_point(aes(color = factor(Cluster))) +
  ggtitle("Clusters on two variables")

ggplot(aes(x = loan_amnt, y = int_rate), data = HierarchicalCluster) + 
  geom_point(aes(color = factor(Cluster))) +
  ggtitle("Clusters on two variables")

The biggest cluster (pink) ranges in the lowest interest rate (about 5%-12.5%) and grades A and B. Most customers in the cluster have a smaller loan amount than those in other clusters. However, the smallest cluster (goldish-color) has the highest interest rate (about 18%-25%), classified as the grade E, F, G, and the customers in the cluster tend to have a larger amount of loan.